In [17]:
import psycopg2

# Connecting to the default 'postgres' database to create a new one
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="orio100%",
    host="localhost",
    port="5432"
)
conn.autocommit = True
cur = conn.cursor()

# Create the new database
cur.execute("CREATE DATABASE Covid19india")

cur.close()
conn.close()
print("Database 'Covid19india' created successfully.")


Database 'Covid19india' created successfully.


In [27]:
import json
import psycopg2

# --- Database connection ---
conn = psycopg2.connect(
    dbname="Covid19india",
    user="postgres",
    password="orio100%",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# --- Create table if not exists ---
cur.execute("""
    CREATE TABLE IF NOT EXISTS covid_cases (
        sno INTEGER PRIMARY KEY,
        date DATE,
        time TEXT,
        state_union_territory TEXT,
        confirmed_indian_national INTEGER,
        confirmed_foreign_national INTEGER,
        cured INTEGER,
        deaths INTEGER,
        confirmed INTEGER
    )
""")
conn.commit()

# Loading JSON data 
with open("covid_19_india.json", "r", encoding="utf-8") as f:
    data = json.load(f)

#  To convert invalid numbers to None 
def parse_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

# Inserting loop with error handling 
failed = []

for row in data:
    try:
        cur.execute("""
            INSERT INTO covid_cases (
                sno, date, time, state_union_territory,
                confirmed_indian_national, confirmed_foreign_national,
                cured, deaths, confirmed
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (sno) DO NOTHING
        """, (
            row.get("Sno"),
            row.get("Date"),
            row.get("Time"),
            row.get("State/UnionTerritory"),
            parse_int(row.get("ConfirmedIndianNational")),
            parse_int(row.get("ConfirmedForeignNational")),
            parse_int(row.get("Cured")),
            parse_int(row.get("Deaths")),
            parse_int(row.get("Confirmed"))
        ))
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"❌ Failed to insert row {row.get('Sno')}: {e}")
        failed.append({"row": row, "error": str(e)})

#  Cleaning up 
cur.close()
conn.close()

# Saving any failed rows for inspection 
if failed:
    with open("failed_rows.json", "w", encoding="utf-8") as f:
        json.dump(failed, f, indent=2)

print("✅ Data upload complete.")
if failed:
    print(f"⚠️ {len(failed)} rows failed. Check 'failed_rows.json' for details.")


❌ Failed to insert row : invalid input syntax for type integer: ""
LINE 6:             ) VALUES ('', '', '', '', NULL, NULL, NULL, NULL...
                              ^

❌ Failed to insert row : invalid input syntax for type integer: ""
LINE 6:             ) VALUES ('', '', '', '', NULL, NULL, NULL, NULL...
                              ^

❌ Failed to insert row : invalid input syntax for type integer: ""
LINE 6:             ) VALUES ('', '', '', '', NULL, NULL, NULL, NULL...
                              ^

❌ Failed to insert row : invalid input syntax for type integer: ""
LINE 6:             ) VALUES ('', '', '', '', NULL, NULL, NULL, NULL...
                              ^

❌ Failed to insert row : invalid input syntax for type integer: ""
LINE 6:             ) VALUES ('', '', '', '', NULL, NULL, NULL, NULL...
                              ^

❌ Failed to insert row : invalid input syntax for type integer: ""
LINE 6:             ) VALUES ('', '', '', '', NULL, NULL, NULL, NULL...
 

In [32]:
import pandas as pd
import psycopg2

#  Database connection 
conn = psycopg2.connect(
    dbname="Covid19india",
    user="postgres",
    password="orio100%",
    host="localhost",
    port="5432"
)

#  Loading data from PostgreSQL into a DataFrame 
query = "SELECT * FROM covid_cases"
df = pd.read_sql_query(query, conn)

#  Saveing DataFrame to CSV 
df.to_csv("covid_19_india.csv", index=False, encoding="utf-8")

# Cleaning up 
conn.close()

print("✅ Data exported to 'covid_19_india.csv'")


✅ Data exported to 'covid_19_india.csv'


/var/folders/0j/gwwj48tx625ddb2ng7nwfsmw0000gn/T/ipykernel_40308/1156474324.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
